# F-strings: A format system to rule them all

La interpolación de cadenas permite embeber código en cadenas de texto usando una sintaxis mínima y limitando al máximo los posibles errores. PEP-498

In [23]:
from datetime import date, datetime, timedelta
talk = 'f-strings'
talk_date = date(2019, 10 , 5)
me = 'juan diego'
minutes_left = 20

print (
  f'Ey Pythonists folks!, today is {talk_date:%A %d %B of %Y}.\n'
  f'I\'m {me.title()}, wellcome to this awesome {talk!r} talk.\nYou\'ll be free '
  f'at {datetime.now() + timedelta(minutes=minutes_left):%H:%M} \U0001f44d'
)

Ey Pythonists folks!, today is Saturday 05 October of 2019.
I'm Juan Diego, wellcome to this awesome 'f-strings' talk.
You'll be free at 20:59 👍


## ¿Cómo se había resuelto esto antes?

Existen tres metodos, que siguen absolutamente vigentes ya que las` f-strings` no suponen la  _deprecración_ de ninguno de los anteriores.

### Printf style formatting

Los objetos tipo String disponen de un operador de [interpolación](https://docs.python.org/3/library/stdtypes.html#printf-style-string-formatting) similar al `sprintf` en **C**.

Si el proceso de _format_ requiere más de un argumento, el valor deberá ser una _tupla_ cuyo número de elementos deberá coincidir con los especificados en la cadena a formatear, o un objeto que mapee _key-value_ (por ejemplo un diccionario)

In [24]:
'%s %s' % ('Old Fashioned', 'formatting')

'Old Fashioned formatting'

In [25]:
'%(style)s %(action)s' % {'style': 'Old Fashioned', 'action': 'formatting'}

'Old Fashioned formatting'

Desventajas: 

Necesita especificar el tipo de conversion:

In [26]:
'%' % 'hello'

ValueError: incomplete format

Solo permite el formateo de enteros, dobles o cadenas. Cualquier otro tipo debe ser convertido a los anteriores antes de aplicar el formato (quizas esto no suponga un problema serio ya que casi cualquier objeto tiene implentado el metodo __str__, __repr__).

Por otro lado existe un problema conocido al pasarle una tupla con mas de un elemento:

In [27]:
'%s' % ('%-formating sucks',  'much')

TypeError: not all arguments converted during string formatting

Podemos evitarlo programando de forma defensiva:

In [28]:
'%s' % (('%-formating sucks',  'much'),)

"('%-formating sucks', 'much')"

O conociendo de antemano el numero de elemento, poco flexible:

In [29]:
'%s %s' % ('%-formating sucks',  'much')

'%-formating sucks much'

### Format String Syntax

Mi [método](https://docs.python.org/3/library/string.html#formatstrings) favorito hasta la llegada de las `f-strings`, de hecho  estas ultimas reutilizan gran parte de la sintaxis y los mecanismos de format pero tambien son mucho mas _verbose_.

Estudiemos este ejemplo:

In [30]:
awesome_conf = 'PyConES'

'This {awesome_conf} is really awesome.'.format(awesome_conf=awesome_conf)

'This PyConES is really awesome.'

Incluso si lo simplificamos al máximo, podemos ver como la variable queda un poco desconectada del contexto lo cual se hace mucho mas evidente al compararlo con el mismo ejemplo usando f-strings:

In [14]:
awesome_conf = 'PyConES'

print('This {} is really awesome.'.format(awesome_conf))

print(f'This {awesome_conf} is really awesome.')


This PyConES is really awesome.
This PyConES is really awesome.


### Template strings

Las [templates ](https://docs.python.org/3/library/string.html#template-strings) se crearon como alternativa al operador de interpolación, que como hemos visto es muy propenso a errores, con el único [objetivo en mente](https://www.python.org/dev/peps/pep-0292/) de simplificar los modos de formatear (en este caso el termino más apropiado sería substituir) cadenas, con el _trade-off_ de sacrificar la expresividad.

In [15]:
from string import Template
Template('Hello from $this').substitute(this='Template')

'Hello from Template'

No soportan el protocolo format, por lo que no es posible realizar conversiones.

El bajo rendimiento y su poca flexibilidad son otros de sus putos débiles.

## F-strings

### ¿Que son?

Una forma de embeber en objetos tipo String expresiones que se evalúan en tiempo de ejecución.

In [16]:
type(f'{type}')

str

### ¿Qué cambios suponen? 

**Ninguno**, los métodos anteriores no han sido deprecados.

### ¿Qué ventajas nos aportan?

Dos fundamentalmente: 

1. Claridad: Sin duda el principal aporte de las `F-strings` es la mejora de la legibilidad  de nuestro código, juzgen ustedes mismos:

In [19]:
ways_2_formatx2 = 6
awesome_lang = 'Python'

print(
    'In %s we have %d ways to do our formatting'
    % (awesome_lang, int(ways_2_formatx2/2))
)
print(
    'In {awesome_lang:s} we have {ways_2_format:d} ways to do our formatting'.format(
        awesome_lang=awesome_lang, ways_2_format=(int(ways_2_formatx2/2))
    )
)
print (f'In {awesome_lang} we have {int(ways_2_formatx2/2)} ways to do our formatting')

In Python we have 3 ways to do our formatting
In Python we have 3 ways to do our formatting
In Python we have 3 ways to do our formatting


2. Rendimiento:  un una `f-string` en primer lugar se evalua la expresión y después se combina con la porción literal para devolver la cadena final. No exsiste nigún otro requerimento, esto las hace muy rapidas y efecientes.

In [ ]:
import timeit

format_funcs = {
    'f-strings': """
def format(superhero, rank):
    return f'{superhero} has a rank of {rank}!'
""",
    '%-formatting': """
def format(superhero, rank):
    return '%s has a rank of %s!' % (superhero, str(rank))
""",
    '.format()': """
def format(superhero, rank):
    return '{} has a rank of {}!'.format(superhero, str(rank))
""",
    'concatenation +': """
def format(superhero, rank):
    return superhero + ' has a rank of ' + str(rank) + '!'
""",
    'concatenation ()': """
def format(superhero, rank):
    return superhero, ' has a rank of ', str(rank), '!'
"""
}

test_func = """def test_format():
    for superhero in ('Wonder Woman', 'Supergirl', 'Batman', 'Robin'):
        for rank in range (1, 101):
            format(superhero, rank)
"""

data = []

for key, func in format_funcs.items():
    data.append({
        'method': key,
        'time': float(timeit.timeit('test_format()', func + test_func, number=10000))
    })

### ¿Como se usan?


Debemos formar una cadena literal a la que se le antepone el prefijo `f` o `F` (ambos son equivalentes).

Por lo demás, su tratamiento es equivalente al de cualquier otra cadena, por ejemplo el carácter que inicia el entrecomillado debe ser igual al que lo finaliza.

Una vez _tokenizado_, un f-string se descomponen en cadenas literales y expresiones, estas ultimas deben de contenerse entre llaves

Para escapar una llave, necesitaremos doblarla `{{` o `}}`.

El caracter de escape `\` no esta permitido dentro de una expresión, este inconveniente puede ser solventado cambiando el carácter de entrecomillado o usando el _triple quoting_.

Opcionalmente y como ultima parte de una expresión se puede especificar un tipo de conversión, con un funcionamiento análogo a `format`: `!s`  llama a `str()`, `!r` a `repr()` y `!a` a `ascii()`.

In [2]:
from datetime import datetime

f'''Playing with {{ {"f-strings '-)".upper()!s} }} {datetime.now():%Y}'''

"Playing with { F-STRINGS '-) } 2019"

También podemos usar F-strings en *modo raw*, añadiendo el prefijo `r` o `R` . De esta forma el carácter de escape `\` no será interpretado.

In [3]:
import re

re.search(fr'=\s*{20 * 2}', 'sum=  40')

<_sre.SRE_Match object; span=(3, 8), match='=  40'>

### Fun with F-strings

Para crear una cadena *multiline*:

In [4]:
print (
    f'F-strings provide a way to embed \'{"expressions"}\' inside string literals, '
    f'using a minimal syntax. '
)

F-strings provide a way to embed 'expressions' inside string literals, using a minimal syntax. 


Ternary operator:

In [5]:
foo = None

f'{foo if foo is not None else "foo"}'

'foo'

## Pitfalls

### Modern Python >= 3.6

En este caso, IMO esto es más una ventaja que un problema, a estas alturas de la película todos deberíamos estar al menos en esta versión de y evitar el *Legacy Python".

### Quoting 

La sintaxis de las F-strings pueden resultar un tanto ardua en lo que se refiere al entrecomillado, de hecho existe una propuesta, la [PEP536](https://www.python.org/dev/peps/pep-0536/) que aboga por su modificación que entre otras cosas permitiría el uso de las comillas dentro de la expresión con independencia de las *exteriores* permitiendo expresiones del tipo: `f'Magic wand: {bag['wand']:^10}'`.

Esta PEP se encuentra en estado *Deferred*, es decir, no hay ningún desarrollador del core que se haya prestado voluntario a desarrollarla.

### Dicts

El uso con diccionarios puede resultar mucho mas *cómodo* con `format`:

In [7]:
nerd = {'name': 'Juan Diego', 'from': 'Almería'}

print('This nerd is {name} from {from}'.format(**nerd))

print(f'This nerd is {nerd["name"]} from {nerd["from"]}')

This nerd is Juan Diego from Almería
This nerd is Juan Diego from Almería


### Logging

Al usar F-strings con logging podemos encontrarno ante un problema de rendimiento ya que llamar de forma automatica al metodo `__str__` del objeto:

In [8]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('fail')

class Dummy:
    def __init__(self, name):
        self._name = name
    def __str__(self):
        print('logging should be >= INFO')
        return self._name
    
c = Dummy('fstring')

logger.debug(f'Created: {c}')
logger.debug('Created: %s', c)

logging should be >= INFO


En caso de que esto suponga un problema resulta más conveniente usar la interpolación tradicional:

### Last but not least

> There should be one-- and preferably only one --obvious way to do it.

Una de las mayores criticas a este nuevo sistemas es que no aporta nada nuevo que no pudiera hacerse con métodos como `format`, y que hace que el lenguaje sea cada vez mas pesado.